In [1]:
# Paqueteo mínimo y compatible (no fijes gradio-client a mano)
# Instalar plotly
!pip install gradio
!pip install gradio plotly -q

In [2]:
import numpy as np, pandas as pd
rng = np.random.default_rng(42)

n = 12000
carreras = [
    "Ingeniería de Sistemas","Administración de Empresas","Psicología",
    "Derecho","Educación Primaria","Contabilidad y Finanzas",
    "Arquitectura","Ciencias de la Comunicación","Medicina Humana",
    "Ingeniería Industrial"
]

# Probabilidades que SUMAN 1.0 exactamente
p = np.array([0.105, 0.105, 0.105, 0.105, 0.10, 0.10, 0.095, 0.095, 0.095, 0.095])

df = pd.DataFrame({
    "alumno_id": np.arange(1, n+1),
    "estructuraalumno": rng.choice(carreras, size=n, p=p),
    "semestre": rng.integers(1, 12, size=n),
})

df["creditosmatriculado"] = rng.choice([12,14,16,18,20,22,24,26,28,30],
                                       size=n, p=[.08,.09,.12,.16,.16,.14,.10,.07,.05,.03])
df["cursosmatriculados"] = np.clip((df["creditosmatriculado"]/4 + rng.normal(0,1,size=n))
                                   .round().astype(int), 3, 10)

# Tasa base por carrera (promedios razonables)
base_aprob = df["estructuraalumno"].map({
    "Psicología":0.77,"Derecho":0.76,"Educación Primaria":0.755,"Contabilidad y Finanzas":0.75,
    "Arquitectura":0.745,"Ciencias de la Comunicación":0.748,"Medicina Humana":0.74,
    "Ingeniería Industrial":0.742,"Ingeniería de Sistemas":0.735,"Administración de Empresas":0.736
}).values

# Ajuste por semestre (ligero)
adj_sem = 0.01*(df["semestre"]>=7).astype(float) - 0.015*(df["semestre"]<=2).astype(float)

# Probabilidad base (antes de inyectar anomalías)
p_apr = np.clip(base_aprob + adj_sem + rng.normal(0,0.03,size=n), 0.55, 0.95)

# ----------------------------------------------------------
# === ANOMALÍAS SINTÉTICAS (para que salten en tus análisis)
# ----------------------------------------------------------
marca = np.array([""]*n, dtype=object)

# 1) Desplazamientos a NIVEL CARRERA (afecta a TODA la carrera)
#    - Ingeniería de Sistemas: caída fuerte (-0.06)  -> debería quedar outlier por abajo
#    - Psicología: subida fuerte (+0.05)              -> outlier por arriba
shift_carrera = {
    "Ingeniería de Sistemas": -0.06,
    "Psicología": +0.05
}
shift_vec = df["estructuraalumno"].map(shift_carrera).fillna(0.0).values
p_apr = p_apr + shift_vec
marca[shift_vec != 0.0] = np.where(shift_vec[shift_vec != 0.0] > 0, "boost_carrera", "drop_carrera")

# 2) Choque por COHORTE local:
#    Medicina Humana en semestre 1 y 2: -0.10 adicional (simula plan de estudios difícil)
mask_mh = (df["estructuraalumno"]=="Medicina Humana") & (df["semestre"]<=2)
p_apr[mask_mh] -= 0.10
marca[mask_mh] = np.where(marca[mask_mh] == "", "choque_mh_s1s2", marca[mask_mh])

# 3) “Grupo problema” en Ingeniería Industrial: 30% de sus alumnos bajan -0.08
mask_ind = (df["estructuraalumno"]=="Ingeniería Industrial")
idx_ind = np.where(mask_ind)[0]
pick = rng.choice(idx_ind, size=int(0.30*len(idx_ind)), replace=False)
p_apr[pick] -= 0.08
marca[pick] = np.where(marca[pick] == "", "grupo_bajo_industrial", marca[pick])

# 4) “Grupo excelencia” en Contabilidad y Finanzas: 25% suben +0.07
mask_cf = (df["estructuraalumno"]=="Contabilidad y Finanzas")
idx_cf = np.where(mask_cf)[0]
pick_cf = rng.choice(idx_cf, size=int(0.25*len(idx_cf)), replace=False)
p_apr[pick_cf] += 0.07
marca[pick_cf] = np.where(marca[pick_cf] == "", "grupo_alto_conta", marca[pick_cf])

# Reclip final de probabilidades
p_apr = np.clip(p_apr, 0.40, 0.98)

# ---------------------------------
# Generación de resultados académicos
# ---------------------------------
df["cursosaprobados"] = [rng.binomial(m, p) for m,p in zip(df["cursosmatriculados"], p_apr)]
df["creditosaprobadostotal"] = (df["cursosaprobados"]*4 + rng.integers(-2,3,size=n)).clip(0)

# Notas relacionadas pero con ruido (coherentes con p_apr)
df["promediosemestre"] = np.clip(rng.normal(12.2 + 5*(p_apr-0.7), 1.4, size=n), 8, 20).round(2)
df["promedioponderado"] = (0.6*df["promediosemestre"] + 0.4*np.clip(
    rng.normal(df["promediosemestre"],1.0,size=n),8,20)).round(2)

# Marca de anomalía para auditoría (útil para validar resultados)
df["marca_anomalia"] = marca

csv_path = "/content/estudiantes_univ.csv"
df.to_csv(csv_path, index=False)
print("CSV creado en:", csv_path, "| filas:", len(df))
print(df["marca_anomalia"].value_counts())

CSV creado en: /content/estudiantes_univ.csv | filas: 12000
marca_anomalia
                         8652
boost_carrera            1268
drop_carrera             1228
grupo_bajo_industrial     336
grupo_alto_conta          302
choque_mh_s1s2            214
Name: count, dtype: int64


In [3]:
# =========================================================
# IMPORTS Y FUNCIONES ORIGINALES (con pequeñas protecciones)
# =========================================================
import io, os, gc
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

def cargar_y_limpiar(file_obj) -> pd.DataFrame:
    df = pd.read_csv(file_obj if isinstance(file_obj, str) else file_obj.name)
    # a numérico seguro
    for c in ["cursosmatriculados","cursosaprobados","creditosmatriculado","creditosaprobadostotal",
              "promediosemestre","promedioponderado","semestre"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    # reglas mínimas
    df = df.dropna(subset=["estructuraalumno","cursosmatriculados","cursosaprobados"]).copy()
    df = df[df["cursosmatriculados"] > 0]
    return df

def _texto_semestres(df: pd.DataFrame) -> str:
    if "semestre" in df.columns and df["semestre"].notna().any():
        return f"{int(df['semestre'].min())} – {int(df['semestre'].max())}"
    return "N/D"

def resumen_texto(df: pd.DataFrame) -> str:
    tasa = (df["cursosaprobados"]/df["cursosmatriculados"]).mean()
    return (
        f"**Registros:** {len(df):,}\n\n"
        f"**Carreras:** {df['estructuraalumno'].nunique()}\n\n"
        f"**Tasa de aprobación global:** {tasa:.2%}\n\n"
        f"**Semestres (min–max):** {_texto_semestres(df)}"
    )

def top8_bar(df: pd.DataFrame) -> str:
    aux = df.copy()
    aux["tasa"] = aux["cursosaprobados"]/aux["cursosmatriculados"]
    tasa_carrera = (
        aux.groupby("estructuraalumno")["tasa"]
           .mean()
           .sort_values(ascending=False)
           .head(8)
           .reset_index()
    )
    tasa_carrera = tasa_carrera[tasa_carrera["tasa"].notna()]

    fig, ax = plt.subplots(figsize=(12,6))
    sns.barplot(data=tasa_carrera, y="estructuraalumno", x="tasa", ax=ax, palette="viridis")
    ax.set_title("Top 8 carreras por tasa de aprobación promedio")
    ax.set_xlabel("Tasa de aprobación"); ax.set_ylabel("Carrera")
    for i, v in enumerate(tasa_carrera["tasa"]):
        ax.text(v, i, f"{v:.2%}", va="center", ha="left")
    plt.tight_layout()

    out = "/content/top8_tasa.png"
    fig.savefig(out, dpi=140, bbox_inches="tight"); plt.close(fig)
    return out

def matriz_indicadores(df: pd.DataFrame) -> pd.DataFrame:
    out = (
        df.assign(tasa=lambda d: d["cursosaprobados"]/d["cursosmatriculados"])
          .groupby("estructuraalumno")
          .agg(
              n=("alumno_id","count") if "alumno_id" in df.columns else ("estructuraalumno","count"),
              tasa_prom=("tasa","mean"),
              cursos_m=("cursosmatriculados","mean"),
              prom_sem=("promediosemestre","mean") if "promediosemestre" in df.columns else ("cursosmatriculados","mean"),
              prom_pond=("promedioponderado","mean") if "promedioponderado" in df.columns else ("cursosmatriculados","mean"),
          )
          .sort_values("tasa_prom", ascending=False)
    )
    return out.round({"tasa_prom":4,"cursos_m":2,"prom_sem":2,"prom_pond":2}).reset_index()

def detectar_outliers_simple(tab: pd.DataFrame,
                             method: str = "iqr",
                             k_iqr: float = 1.5,
                             z_thr: float = 2.0,
                             mad_thr: float = 3.5,
                             min_n: int = 50) -> pd.DataFrame:
    """
    Marca anomalias en 'tasa_prom' por carrera.
    - method: "iqr" | "z" | "mad"
    - min_n: mínimo de registros por carrera para ser evaluada
    """
    out = tab.copy()
    out["anomalia"] = ""
    # filtra carreras con suficiente tamaño muestral si existe 'n'
    mask_eval = out["n"] >= min_n if "n" in out.columns else np.ones(len(out), dtype=bool)
    s = pd.to_numeric(out.loc[mask_eval, "tasa_prom"], errors="coerce").dropna()

    # si no hay datos suficientes o no hay variación, no marcamos nada
    if len(s) < 4 or s.nunique() < 2:
        return out

    if method.lower() == "iqr":
        q1, q3 = s.quantile([0.25, 0.75])
        iqr = q3 - q1
        # si iqr≈0, cae a z-score para evitar “todo vacío”
        if iqr <= 1e-9:
            m, sd = s.mean(), s.std(ddof=1)
            if sd <= 1e-9:  # sin variación
                return out
            flags = (np.abs((out["tasa_prom"] - m) / sd) > z_thr) & mask_eval
        else:
            low, high = q1 - k_iqr * iqr, q3 + k_iqr * iqr
            flags = ((out["tasa_prom"] < low) | (out["tasa_prom"] > high)) & mask_eval

    elif method.lower() == "z":
        m, sd = s.mean(), s.std(ddof=1)
        if sd <= 1e-9:
            return out
        flags = (np.abs((out["tasa_prom"] - m) / sd) > z_thr) & mask_eval

    else:  # "mad" robusto
        med = s.median()
        mad = (np.abs(s - med)).median()
        if mad <= 1e-9:
            return out
        z_mad = 0.6745 * (out["tasa_prom"] - med) / mad
        flags = (np.abs(z_mad) > mad_thr) & mask_eval

    out.loc[flags, "anomalia"] = "⚠️"
    return out

def pipeline(file_obj):
    df = cargar_y_limpiar(file_obj)
    txt = resumen_texto(df)
    img = top8_bar(df)
    tabla = detectar_outliers_simple(matriz_indicadores(df))
    gc.collect()
    return txt, img, tabla


# =========================================================
# VISUALIZACIONES EXTRA (compatibles con tu flujo)
# =========================================================

def _filtrar_base(df, carreras_sel=None, sem_min=1, sem_max=12):
    g = df.copy()
    for c in ["cursosaprobados","cursosmatriculados","creditosmatriculado","creditosaprobadostotal","semestre"]:
        if c in g.columns:
            g[c] = pd.to_numeric(g[c], errors="coerce")
    g = g.dropna(subset=["cursosaprobados","cursosmatriculados"])
    g = g[g["cursosmatriculados"] > 0]
    if "semestre" in g.columns:
        smin = int(min(sem_min, sem_max)); smax = int(max(sem_min, sem_max))
        g = g[(g["semestre"] >= smin) & (g["semestre"] <= smax)]
    if carreras_sel:
        g = g[g["estructuraalumno"].astype(str).isin(carreras_sel)]
    g["tasa"] = np.divide(g["cursosaprobados"], g["cursosmatriculados"], where=g["cursosmatriculados"]>0)
    return g

def _msg_fig(texto: str, path="/content/viz_msg.png"):
    fig, ax = plt.subplots(figsize=(10,2.8))
    ax.axis("off")
    ax.text(0.5, 0.5, texto, ha="center", va="center", fontsize=14)
    fig.savefig(path, dpi=140, bbox_inches="tight"); plt.close(fig)
    return path

def plot_extra_top_barras(df, topn=8, carreras_sel=None, sem_min=1, sem_max=12):
    g = _filtrar_base(df, carreras_sel, sem_min, sem_max)
    if g.empty:
        return _msg_fig("Sin datos para los filtros seleccionados.", "/content/extra_topN.png")
    top = (g.groupby("estructuraalumno")["tasa"].mean()
             .sort_values(ascending=False).head(int(topn)).reset_index())
    fig, ax = plt.subplots(figsize=(12,6))
    sns.barplot(data=top, y="estructuraalumno", x="tasa", ax=ax, palette="viridis")
    ax.set_title(f"Top {len(top)} carreras por tasa de aprobación promedio")
    ax.set_xlabel("Tasa de aprobación"); ax.set_ylabel("Carrera")
    for i, v in enumerate(top["tasa"]): ax.text(v, i, f"{v:.2%}", va="center", ha="left")
    plt.tight_layout()
    out = "/content/extra_topN.png"
    fig.savefig(out, dpi=140, bbox_inches="tight"); plt.close(fig)
    return out

def plot_extra_histograma(df, feature="tasa", bins=20, carreras_sel=None, sem_min=1, sem_max=12):
    g = _filtrar_base(df, carreras_sel, sem_min, sem_max)
    if g.empty:
        return _msg_fig("Sin datos para los filtros seleccionados.", "/content/extra_hist.png")
    feat = "tasa" if feature == "tasa" else feature
    if feat not in g.columns:
        return _msg_fig(f"No existe la columna '{feature}' en el dataset.", "/content/extra_hist.png")
    fig, ax = plt.subplots(figsize=(12,5))
    sns.histplot(g[feat].dropna(), bins=int(bins), ax=ax)
    ax.set_title(f"Histograma de {feat}")
    ax.set_xlabel(feat); ax.set_ylabel("Frecuencia")
    plt.tight_layout()
    out = "/content/extra_hist.png"
    fig.savefig(out, dpi=140, bbox_inches="tight"); plt.close(fig)
    return out

def plot_extra_scatter(df, sample=5000, carreras_sel=None, sem_min=1, sem_max=12):
    g = _filtrar_base(df, carreras_sel, sem_min, sem_max)
    if g.empty:
        return _msg_fig("Sin datos para los filtros seleccionados.", "/content/extra_scatter.png")
    # eje X: preferir creditosmatriculado, si no existe usar cursosmatriculados
    xcol = "creditosmatriculado" if "creditosmatriculado" in g.columns else "cursosmatriculados"
    if len(g) > sample:
        g = g.sample(int(sample), random_state=42)
    fig, ax = plt.subplots(figsize=(12,6))
    ax.scatter(g[xcol], g["tasa"], alpha=0.25, s=10)
    ax.set_title(f"Dispersión: {xcol} vs tasa de aprobación (muestra)")
    ax.set_xlabel(xcol); ax.set_ylabel("Tasa de aprobación")
    ax.grid(True, alpha=0.25)
    plt.tight_layout()
    out = "/content/extra_scatter.png"
    fig.savefig(out, dpi=140, bbox_inches="tight"); plt.close(fig)
    return out

def plot_extra_heatmap(df, carreras_sel=None, sem_min=1, sem_max=12):
    g = _filtrar_base(df, carreras_sel, sem_min, sem_max)
    if g.empty:
        return _msg_fig("Sin datos para los filtros seleccionados.", "/content/extra_heatmap.png")
    if "creditosmatriculado" not in g.columns:
        return _msg_fig("No hay 'creditosmatriculado' para el heatmap.", "/content/extra_heatmap.png")
    bins = [0, 9, 12, 15, 18, 21, 24, 27, np.inf]
    labels = ["≤9","10–12","13–15","16–18","19–21","22–24","25–27","≥28"]
    g["_bin_cred"] = pd.cut(pd.to_numeric(g["creditosmatriculado"], errors="coerce"),
                            bins=bins, labels=labels, include_lowest=True)
    tab = (g.groupby(["estructuraalumno","_bin_cred"], observed=False)["tasa"]
             .mean().unstack().reindex(columns=labels))
    if tab.notna().sum().sum() == 0:
        return _msg_fig("No hay promedios válidos para el heatmap.", "/content/extra_heatmap.png")
    fig, ax = plt.subplots(figsize=(14,8))
    sns.heatmap(tab, annot=True, fmt=".2f", cmap="viridis",
                linewidths=.5, cbar_kws={'label':'Tasa prom.'}, vmin=0, vmax=1, ax=ax)
    ax.set_title("Tasa por carrera × tramo de créditos")
    ax.set_xlabel("Créditos matriculados"); ax.set_ylabel("Carrera")
    plt.tight_layout()
    out = "/content/extra_heatmap.png"
    fig.savefig(out, dpi=140, bbox_inches="tight"); plt.close(fig)
    return out

def run_viz_extra(file_obj,
                  chart_kind="Top-N (barras)",
                  topn=8, bins=20, sample_scatter=5000,
                  carreras_sel=None, sem_min=1, sem_max=12):
    """
    chart_kind ∈ {"Top-N (barras)", "Histograma (tasa)", "Dispersión", "Heatmap"}
    Devuelve el path del PNG generado (si no hay datos, devuelve PNG con mensaje).
    """
    df = cargar_y_limpiar(file_obj)
    if chart_kind == "Top-N (barras)":
        return plot_extra_top_barras(df, topn=topn, carreras_sel=carreras_sel,
                                     sem_min=sem_min, sem_max=sem_max)
    elif chart_kind == "Histograma (tasa)":
        return plot_extra_histograma(df, feature="tasa", bins=bins, carreras_sel=carreras_sel,
                                     sem_min=sem_min, sem_max=sem_max)
    elif chart_kind.startswith("Dispersión"):
        return plot_extra_scatter(df, sample=sample_scatter, carreras_sel=carreras_sel,
                                  sem_min=sem_min, sem_max=sem_max)
    else:
        return plot_extra_heatmap(df, carreras_sel=carreras_sel, sem_min=sem_min, sem_max=sem_max)

In [4]:
import gradio as gr
import pandas as pd

# ---- helpers para poblar opciones dinámicas desde el CSV ----
def _leer_head(file_obj, max_rows=200_000):
    df = pd.read_csv(file_obj.name if hasattr(file_obj, "name") else file_obj,
                     nrows=max_rows)
    return df

def get_meta_from_file(file_obj):
    """Devuelve lista de carreras únicas y rango de semestres (min, max)."""
    try:
        df = _leer_head(file_obj)
        carreras = sorted(map(str, df["estructuraalumno"].dropna().astype(str).unique())) \
                   if "estructuraalumno" in df.columns else []
        if "semestre" in df.columns and pd.to_numeric(df["semestre"], errors="coerce").notna().any():
            sem_min = int(pd.to_numeric(df["semestre"], errors="coerce").min())
            sem_max = int(pd.to_numeric(df["semestre"], errors="coerce").max())
        else:
            sem_min, sem_max = 1, 12
        return carreras, sem_min, sem_max
    except Exception:
        return [], 1, 12


with gr.Blocks(title="Análisis de Tasa de Estudiantes") as app:
    gr.Markdown(
        "# Data Mining para patrones anómalos en la tasa de estudiantes universitarios\n"
        "Carga tu CSV y explora indicadores, **top 8** por tasa y visualizaciones extra."
    )

    # ========= Entrada común =========
    with gr.Row():
        inp = gr.File(label="Sube tu CSV (o usa el que te di)", file_types=[".csv"])

    # guardamos metadatos del archivo para poblar controles dinámicos
    carreras_state = gr.State([])
    sem_min_state  = gr.State(1)
    sem_max_state  = gr.State(12)

    # al subir archivo, calculamos opciones de carreras y rango de semestres
    def on_file(file_obj):
        carreras, a, b = get_meta_from_file(file_obj)
        return carreras, a, b, gr.update(choices=carreras)
    carreras_dd_for_update = gr.CheckboxGroup(choices=[], visible=False)  # dummy receptor

    inp.change(
        on_file,
        inputs=inp,
        outputs=[carreras_state, sem_min_state, sem_max_state, carreras_dd_for_update],
        queue=False,
    )

    with gr.Tabs():
        # ========= TAB 1: flujo original =========
        with gr.Tab("Análisis base"):
            with gr.Row():
                btn = gr.Button("Analizar", variant="primary")
            with gr.Row():
                out_txt = gr.Markdown(label="Resumen")
            with gr.Row():
                out_img = gr.Image(label="Top 8 por tasa",
                                   show_download_button=True, type="filepath")
            with gr.Row():
                out_tab = gr.Dataframe(
                    label="Indicadores por carrera (con flag de anormalidad)",
                    interactive=False, wrap=True
                )

            btn.click(pipeline, inputs=inp, outputs=[out_txt, out_img, out_tab])

        # ========= TAB 2: visualizaciones extra =========
        with gr.Tab("Visualizaciones extra"):
            gr.Markdown("Elige el tipo de gráfico y los filtros.")

            with gr.Row():
                chart_kind = gr.Radio(
                    ["Top-N (barras)", "Histograma (tasa)", "Dispersión", "Heatmap"],
                    value="Top-N (barras)", label="Tipo de gráfico"
                )
                topn  = gr.Slider(2, 20, value=8, step=1, label="Top-N (para barras)")
                bins  = gr.Slider(5, 60, value=20, step=1, label="Bins (para histograma)")
                samp  = gr.Slider(1000, 30000, value=5000, step=500,
                                  label="Muestra (para dispersión)")

            with gr.Row():
                carreras_sel = gr.CheckboxGroup(
                    choices=[], label="Filtrar carreras (opcional)"
                )
                sem_min = gr.Slider(1, 20, value=1, step=1, label="Semestre mínimo")
                sem_max = gr.Slider(1, 20, value=12, step=1, label="Semestre máximo")

            # cuando subes el archivo, también actualizamos estos controles visibles
            def fill_controls(file_obj):
                carreras, a, b = get_meta_from_file(file_obj)
                return (gr.update(choices=carreras),
                        gr.update(value=a),
                        gr.update(value=b))
            inp.change(fill_controls, inputs=inp,
                       outputs=[carreras_sel, sem_min, sem_max], queue=False)

            gen = gr.Button("Generar gráfico", variant="secondary")
            viz_img = gr.Image(label="Visualización", show_download_button=True, type="filepath")

            def _run_viz(file_obj, kind, n, bns, sample, carreras_list, smin, smax):
                carreras_list = carreras_list or None  # None si lista vacía
                return run_viz_extra(
                    file_obj,
                    chart_kind=kind,
                    topn=int(n),
                    bins=int(bns),
                    sample_scatter=int(sample),
                    carreras_sel=carreras_list,
                    sem_min=int(min(smin, smax)),
                    sem_max=int(max(smin, smax)),
                )

            gen.click(
                _run_viz,
                inputs=[inp, chart_kind, topn, bins, samp, carreras_sel, sem_min, sem_max],
                outputs=viz_img
            )

    # En Colab usa share=True; show_api=False para UI más limpia.
    app.launch(share=True, show_api=False, debug=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://99071d7893b864bfe3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
